<a href="https://colab.research.google.com/github/QorbQuant/Crypto-tax-data/blob/main/HarmonyTransform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import pandas as pd
import numpy as np

from google.colab import data_table 
data_table.enable_dataframe_formatter()


In [92]:
# inputs

file = '/content/export_transaction_0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083 (3).csv'
wallet = "0x4bf44Bc1da6dbf4277C3ac3F69C0809BF9B3a083".lower()

stably_wallets = ['0x4bf44Bc1da6dbf4277C3ac3F69C0809BF9B3a083', '0xb27Dc47531a4E8472116A8672CC29041EDEf27aE', '0x64eBF628a6cE6D9719e54F1db9377E5E231044b3', \
                  '0x664FE9758008B398Dd5366BeCf07cF344ae5CC52', '0x4eE18C6942B4f90AFa0d8A5C5dAADD1Fa706a61A', '0x512B9120A1a67ab71F0E3f5F3D0b3A824A204ddd', \
                  '0x764ED0Ec8708D80c48EAc0bD8cB2B80eA2c43c8C']

stably_wallets = list(map(str.lower, stably_wallets))
stably_wallets = [item for item in stably_wallets if item != wallet]


In [93]:
df = pd.read_csv(file)

In [94]:
df.columns

Index(['Txhash', ' Blockno', ' UnixTimestamp', ' DateTime', ' From', ' To',
       ' Value_IN(ONE)', ' Value_OUT(ONE)', ' CurrentValue @ $0.01626218/ONE',
       ' TxnFee(ONE)', ' TxnFee(USD)', ' Method'],
      dtype='object')

In [95]:
df.dtypes

Txhash                              object
 Blockno                             int64
 UnixTimestamp                       int64
 DateTime                           object
 From                               object
 To                                 object
 Value_IN(ONE)                     float64
 Value_OUT(ONE)                      int64
 CurrentValue @ $0.01626218/ONE    float64
 TxnFee(ONE)                       float64
 TxnFee(USD)                       float64
 Method                             object
dtype: object

In [96]:
# strip white spaces
df[' To'] = df[' To'].str.strip().str.lower()
df[' From'] = df[' From'].str.strip().str.lower()

# conditions
df['transactionType'] = np.where(df[' To'] == wallet, 'deposit', 'withdrawal')
df['amount'] = df.apply(lambda row: row[' Value_OUT(ONE)'] if row['transactionType'] == 'withdrawal' else row[' Value_IN(ONE)'], axis=1)
df['fee'] = np.where(df['transactionType'] == 'withdrawal', df[' TxnFee(ONE)'], 0)


In [97]:
# deterimine if tax exempt
condition_1 = (df['transactionType'] == 'withdrawal') & (df[' To'].isin(stably_wallets))
condition_2 = (df['transactionType'] == 'deposit') & (df[' From'].isin(stably_wallets))

df['taxExempt'] = 'FALSE'
df.loc[condition_1 | condition_2, 'taxExempt'] = 'TRUE'

In [98]:
# rename columns
df = df.rename(columns={' DateTime': 'time','Txhash': 'blockchainId', ' From': 'fromAddress',' To': 'toAddress'})


In [99]:
# add blank columns for csv
df['id'] = ''
df['remoteContactId'] = ''
df['amountTicker'] = 'ONE'
df['cost'] = ''
df['costTicker'] = ''
df['feeTicker'] = 'ONE'
df['memo'] = ''
df['accountId'] = ''
df['contactId'] = ''
df['categoryId'] = ''
df['tradeId'] = ''
df['description'] = ''
df['groupId'] = ''

In [100]:
final_df = df[['id', 'remoteContactId', 'amount', 'cost', 'costTicker', 'fee', 'feeTicker', 'time', \
              'blockchainId', 'memo', 'transactionType', 'accountId', 'contactId', 'categoryId',   \
              'taxExempt', 'tradeId', 'description', 'fromAddress', 'toAddress', 'groupId']]

final_df

,id,remoteContactId,amount,cost,costTicker,fee,feeTicker,time,blockchainId,memo,transactionType,accountId,contactId,categoryId,taxExempt,tradeId,description,fromAddress,toAddress,groupId
0,,,0.000000e+00,,,0.0,ONE,2022-12-29 17:12:09,0xf6f7d929e700cdcf1221ddba9383db65b81107037ccb...,,deposit,,,,TRUE,,,0xb27dc47531a4e8472116a8672cc29041edef27ae,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
1,,,0.000000e+00,,,0.0,ONE,2022-12-29 17:12:06,0xb8059895a05f9502a49fdaa1e2351c4d174400a99543...,,deposit,,,,TRUE,,,0xb27dc47531a4e8472116a8672cc29041edef27ae,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
2,,,0.000000e+00,,,0.0,ONE,2022-12-29 17:12:43,0xeceb5fa6e99a109f2987e322f373089d58af359e10b9...,,deposit,,,,TRUE,,,0xb27dc47531a4e8472116a8672cc29041edef27ae,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
3,,,1.836885e+06,,,0.0,ONE,2022-11-22 11:11:46,0x402f484a6634b234197d6e15d6e02865150125b4cf20...,,deposit,,,,FALSE,,,0xed1229b9931c5396975ed22aae78117e06bdeed6,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
4,,,0.000000e+00,,,0.0,ONE,2022-11-14 14:11:03,0x395a4b0f67e1434a15f93d07214d9844968267e8a42f...,,deposit,,,,TRUE,,,0xb27dc47531a4e8472116a8672cc29041edef27ae,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
5,,,0.000000e+00,,,0.0,ONE,2022-11-08 10:11:43,0xa96dcc0b3a98ece2acc4c30ec4260d166dc5eda725d0...,,deposit,,,,TRUE,,,0xb27dc47531a4e8472116a8672cc29041edef27ae,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
6,,,1.154201e+06,,,0.0,ONE,2022-11-04 17:11:37,0x6fb9ee865c19cd74fdf41a9ebd590dafb1f2d9db8859...,,deposit,,,,FALSE,,,0xed1229b9931c5396975ed22aae78117e06bdeed6,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
7,,,1.226693e+06,,,0.0,ONE,2022-11-03 13:11:04,0x78265c208602c74e4b34b6bbef7656034abd30312e9f...,,deposit,,,,FALSE,,,0xed1229b9931c5396975ed22aae78117e06bdeed6,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
8,,,0.000000e+00,,,0.0,ONE,2022-08-27 13:08:53,0x832eb2583de0113ec5790ee5eed24e87496bf442b777...,,deposit,,,,TRUE,,,0xb27dc47531a4e8472116a8672cc29041edef27ae,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,
9,,,0.000000e+00,,,0.0,ONE,2022-08-27 13:08:49,0x43340009871f27d0ac35029f3f7d298241e728bbbc84...,,deposit,,,,TRUE,,,0xb27dc47531a4e8472116a8672cc29041edef27ae,0x4bf44bc1da6dbf4277c3ac3f69c0809bf9b3a083,


In [101]:
final_df.to_csv('harmony_transactions.csv', index=False)